In [42]:
import torch
import torch.nn as nn
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from enum import Enum
from functools import cached_property
from mubench.base.model import DNN
import random
np.random.seed(0)
random.seed(0)


In [43]:
dnn = DNN(200, 2, 1)

In [51]:
class tt(str, Enum):
    x = 'a'
    y = 'b'

    @classmethod
    def get_options(cls):
        options = list()
        for k, v in cls.__dict__.items():
            if not k.startswith('_') and k != 'get_options':
                options.append(v.value)
        return options

In [53]:
tt('a') == tt.x

True

In [54]:
tt.get_options()

['a', 'b']

In [55]:
tt.x in tt.get_options()

True

In [56]:
torch.save(tt.x, 'x.pt')

In [57]:
ttt = torch.load('x.pt')

In [58]:
ttt

<tt.x: 'a'>